In [26]:
class TreeNode(object):
    """A node in the PrefixTree."""
    
    __slots__ = ['word_ids', 'tree']
    
    def __init__(self):
        self.word_ids = []  # ids of words with the substring up to this node
        self.tree = {}  # map from char to child TreeNode(s)
    
    def __setitem__(self, char, node):
        self.tree[char] = node
    
    def __getitem__(self, char):
        return self.tree[char]


class PrefixTree(object):
    """Prefix tree for simple auto-completion."""
    
    def __init__(self, words=None):
        self.word2id = {}
        self.id2word = []
        self.tree = {}
        
        if words is not None:
            self.insert_all(words)
    
    def insert(self, word):
        """Insert a word into the PrefixTree if it has not already been inserted.
        The word is assigned an ID (auto-incrementing non-negative int).
        This ID is then stored at each node in the path defined by its characters.
        
        """
        if word in self.word2id:
            return  # word already in tree
        
        word_id = self._add_word(word)
        node = self.tree
        for char in word:
            try:
                node = node[char]
            except KeyError:
                new_node = TreeNode()
                node[char] = new_node
                node = new_node
                
            node.word_ids.append(word_id)
                
    def _get_word_id(self, word):
        try:
            return self.word2id[word]
        except KeyError:
            return self._add_word(word)
        
    def _add_word(self, word):
        word_id = len(self.word2id)
        self.word2id[word] = word_id
        self.id2word.append(word)
        return word_id

    def insert_all(self, words):
        for word in words:
            self.insert(word)
    
    def search(self, substr):
        """Return all words that start with this substring.
        The `substr` is stripped of whitespace and lowercased before searching.
        
        """
        substr = substr.strip().lower()
        try:
            node = self.tree[substr[0]]
        except KeyError:
            return []
        
        for char in substr[1:]:
            try:
                node = node[char]
            except KeyError:
                return []
                
        return [self.id2word[word_id] for word_id in node.word_ids]


In [27]:
tree = PrefixTree()

tree.insert('hello')
tree.insert('helios')
tree.insert('howdy')
tree.insert('house')
assert(tree.search('he') == ['hello', 'helios'])
assert(tree.search('hel') == ['hello', 'helios'])
assert(tree.search('ho') == ['howdy', 'house'])
assert(tree.search('hi') == [])

tree.insert('foo')
tree.insert('fizz')
assert(tree.search('fo') == ['foo'])
tree.insert('foobar')
assert(tree.search('foo') == ['foo', 'foobar'])

del tree

In [30]:
# Read in a file with many english words
with open('cleaned-words.txt') as f:
    words = [l.strip() for l in f if l.strip()]

print("There are %d words" % len(words))

There are 354909 words


In [29]:
# How long does it take to insert all the words?
%timeit PrefixTree(words)

1 loop, best of 3: 4.59 s per loop


In [32]:
# How long does it take to conduct 1 million searches?
import random
random.seed(42)

# Let's grab a bunch of random substrings to search for.
substrings = []
num_searches = 1000000
while len(substrings) < num_searches:
    for word in words:
        if len(word) <= 2:
            substrings.append(word)
        else:
            offset = random.randint(2, len(word) - 1)
            substrings.append(word[offset:])

In [34]:
# The complexity should be linear in the number of characters + words in the subtree.
# Once the node is found, the words are looked up by the ids stored there.
tree = PrefixTree(words)
%timeit map(tree.search, substrings)

1 loop, best of 3: 15min 38s per loop


In [42]:
# My timings were 4.59s for 354,909 words and 15 min, 38s for 1M lookups.
insert_time_s = (4.59 / 354909)
print("Avg insertion time/word: %.4fms" % (insert_time_s * 1000))
lookup_time_s = ((15 * 60) + 38) / 1000000.
print("Avg lookup time: %.4fms" % (lookup_time_s * 1000))

Avg insertion time/word: 0.0129ms
Avg lookup time: 0.9380ms
